In [17]:
# -*- coding: utf-8 -*-
import os
import re
from html.parser import HTMLParser
from bs4 import BeautifulSoup
from bs4 import element

# Functions

In [18]:
def get_files(base_dir,file_list):
    files = os.listdir(base_dir)
    for i in files:
        abs_path = os.path.join(base_dir,i)
        if re.match(r'(.*)PMC(.*).html',abs_path) and ('highlighted' not in abs_path):
            file_list.append(abs_path)
        elif os.path.isdir(abs_path)&('ipynb_checkpoints' not in abs_path):
            get_files(abs_path,file_list)
    return file_list

def extract_all_h2_headings(file_list_without_duplicates):
    heading_text_list=[]
    heading_text_list_with_filename=[]
    for file in file_list_without_duplicates:
        with open(file,'r',encoding='UTF-8',errors='ignore') as f:
            text = f.read()
            soup = BeautifulSoup(text, 'html.parser')
            headings = soup.find_all('h2')
            for number, heading in enumerate(headings):
                heading_text =  heading.get_text()
                
                # [heading starts with "1.",heading starts with "P1"]
                pattern_list=['^(([0-9]\. *)+)','^(P[0-9])','^([0-9]) ']
                pattern_status=False
                for pattern in pattern_list:
                    if re.search(pattern,heading_text):
                        pattern_status=True
                        heading_text_list.append(heading_text.strip(re.search(pattern,heading_text).group()))
                        heading_text_list_with_filename.append(heading_text.strip(re.search(pattern,heading_text).group())+'thereisaspace'+file.split('/')[-1])

                if not pattern_status:
                    heading_text_list.append(heading_text)
                    heading_text_list_with_filename.append(heading_text+'thereisaspace'+file.split('/')[-1])
                
    heading_text_list_tmp=[i.lower().strip('.').strip(' ').strip(':').strip(' ').strip('*').strip(':').replace('\n',' ') for i in heading_text_list]
    heading_text_list = heading_text_list_tmp
    heading_text_list_with_filename_tmp=[i.lower().strip('.').strip(' ').strip(':').strip(' ').strip('*').strip(':').replace('\n',' ') for i in heading_text_list_with_filename ]
    heading_text_list_with_filename=heading_text_list_with_filename_tmp
    return heading_text_list, heading_text_list_with_filename


def keep_count_unique_heading(heading_text_list):
    heading_text_list_unique=[]
    heading_text_list_unique_for_count=[]
    for number,heading in enumerate(heading_text_list):
        number_of_heading = heading_text_list.count(heading)
        if number_of_heading >1 and heading_text_list_unique_for_count.count(heading)<1: 
            heading_text_list_unique.append(heading_text_list_with_filename[number]+'\t'+str(number_of_heading))
            heading_text_list_unique_for_count.append(heading)
    return heading_text_list_unique, heading_text_list_unique_for_count

# Get file list

In [20]:
if __name__=='__main__':
    base_dir = input('Input base directory:')
    file_list = []
    file_list = get_files(base_dir,file_list)
    file_list_without_duplicates=[]
    for file in file_list:
        PMC_number=file.split('/')[-1]
        duplicates_status=False
        for f in file_list_without_duplicates:
            if PMC_number in f:
                duplicates_status=True
        if not duplicates_status:
            file_list_without_duplicates.append(file)

Input base directory: /rdsgpfs/general/user/yh4218/home/data/NLP


# All headings

In [13]:
    heading_text_list, heading_text_list_with_filename = extract_all_h2_headings(file_list_without_duplicates)

# Unique headings

In [14]:
    heading_text_list_unique, heading_text_list_unique_for_count = keep_count_unique_heading(heading_text_list)

In [15]:
    with open('/rdsgpfs/general/user/yh4218/home/data/all_unique_h2_headings_with_counts.txt','w') as f:
        for heading in heading_text_list_unique:
            f.write(str(heading)+'\n')